In [1]:
# импорт требуемых библиотек
import pandas as pd
import requests
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/cld-21
import some_funcs

Mounted at /content/drive
/content/drive/MyDrive/cld-21


In [2]:
# создание pandas df из csv
hits_df = pd.read_csv('metrika_cloud_case_data_hits.csv', sep = '\t')
visits_df = pd.read_csv('metrika_cloud_case_data_visits.csv', sep = '\t')

In [3]:
# переменные для подключения к clickhouse
CH_HOST_NAME = 'rc1a-rico851cbhrudsig.mdb.yandexcloud.net'
CH_USER      = 'manokhin'
CH_DB_NAME   = 'metrica_data'
CH_PASS      = open('chpass.txt').read().strip()
CH_HOST      = f'https://{CH_HOST_NAME}:8443'
CH_CASERT    = 'YandexInternalRootCA.crt'

In [4]:
# подключение к clickhouse
from some_funcs import simple_ch_client
my_client = simple_ch_client(CH_HOST, CH_USER, CH_PASS, CH_CASERT)
my_client.get_version()

23.3.7.5



In [5]:
# загрузка hits
hits_df.rename(columns={'ym:pv:browser':'Browser',
                'ym:pv:clientID':'ClientID',
                'ym:pv:date':'EventDate',
                'ym:pv:dateTime':'EventTime',
                'ym:pv:deviceCategory':'DeviceCategory',
                'ym:pv:lastTrafficSource':'TraficSource',
                'ym:pv:operatingSystemRoot':'OSRoot',
                'ym:pv:URL':'URL'}, inplace = True)

q = f'drop table if exists {CH_DB_NAME}.hits '
my_client.get_clickhouse_data(q)

q = f'''
create table {CH_DB_NAME}.hits (
    Browser String,
    ClientID UInt64,
    EventDate Date,
    EventTime DateTime,
    DeviceCategory String,
    TraficSource String,
    OSRoot String,
    URL String
) ENGINE = MergeTree() ORDER BY (intHash32(ClientID), EventDate) SAMPLE BY intHash32(ClientID) SETTINGS index_granularity=8192
'''

my_client.get_clickhouse_data(q)

my_client.upload(
    f'{CH_DB_NAME}.hits',
    hits_df.to_csv(index = False, sep = '\t'))

''

In [6]:
# загрузка visits
visits_df.rename(columns={'ym:s:browser':'Browser',
                'ym:s:clientID':'ClientID',
                'ym:s:date':'StartDate',
                'ym:s:dateTime':'StartTime',
                'ym:s:deviceCategory':'DeviceCategory',
                'ym:s:lastTrafficSource':'TraficSource',
                'ym:s:operatingSystemRoot':'OSRoot',
                'ym:s:purchaseRevenue': 'Purchase.Revenue',
                'ym:s:purchaseID': 'Purchase.ID',
                'ym:s:startURL':'StartURL'}, inplace = True)

visits_df['Purchases'] = visits_df['Purchase.Revenue'].map(lambda x: x.count(',') + 1 if x != '[]' else 0 )
visits_df['Revenue'] = visits_df['Purchase.Revenue'].map(lambda x: sum(map(int,x[1:-1].split(','))) if x != '[]' else 0)

visits_df.drop(columns=['Purchase.ID','Purchase.Revenue'], inplace=True)

q = f'drop table if exists {CH_DB_NAME}.visits '
my_client.get_clickhouse_data(q)

q = f'''
create table {CH_DB_NAME}.visits (
    Browser String,
    ClientID UInt64,
    StartDate Date,
    StartTime DateTime,
    DeviceCategory UInt8,
    TraficSource String,
    OSRoot String,
    Purchases Int32,
    Revenue Double,
    StartURL String
) ENGINE = MergeTree() ORDER BY (intHash32(ClientID), StartDate) SAMPLE BY intHash32(ClientID) SETTINGS index_granularity=8192
'''

my_client.get_clickhouse_data(q)

my_client.upload(
    f'{CH_DB_NAME}.visits',
    visits_df.to_csv(sep='\t', index =False))

''